In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from numpy import argmax
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import (
    Activation,
    Convolution2D,
    Dense,
    Dropout,
    Flatten,
    MaxPooling2D,
)
from tensorflow.keras.models import Sequential
import cv2 as cv
import numpy as np
import os
from tqdm import tqdm
import random
from tensorflow.keras.callbacks import TensorBoard

In [2]:
CATEGORIES = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
train_ds = []

IMG_SIZE = 128
batch_size = 50


def create_training_data():
    DATADIR = "/home/karol/python_projekty/cv2/sudoku/data/train"
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in tqdm(os.listdir(path)):
            img_array = cv.imread(os.path.join(path, img))
            gray = cv.cvtColor(img_array, cv.COLOR_RGB2GRAY)
            ret, thresh = cv.threshold(gray, 127, 255, cv.THRESH_BINARY_INV)
            train_ds.append([thresh, class_num])


create_training_data()
print(len(train_ds))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 12434.01it/s]

10000


In [3]:
random.shuffle(train_ds)

In [4]:
X = []
y = []

for features, label in train_ds:
    X.append(features)
    y.append(label)

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y).reshape(-1, 1)

[[[[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  ...

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]]]


In [6]:
X = X / 255.0

In [7]:
X.shape

(10000, 128, 128, 1)

In [8]:
layers = [
    Convolution2D(128, 3, 3, input_shape=X.shape[1:], activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),
    Convolution2D(64, 2, 1, activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),
    Convolution2D(32, 2, 1, activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(1024, activation="relu"),
    Dropout(0.5),
    Dense(512, activation="relu"),
    Dropout(0.5),
    Dense(10),
    Activation("sigmoid"),
]

model = Sequential()
for layer in layers:
    model.add(layer)

In [9]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "/home/karol/python_projekty/cv2/sudoku/models/save_at_{epoch}.h5"
    ),
]
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)
model.fit(np.array(X), np.array(y), batch_size=32, epochs=10)

Epoch 1/10
313/313 [==============================] - 4s 8ms/step - loss: 0.4177 - accuracy: 0.8628
Epoch 2/10
313/313 [==============================] - 2s 8ms/step - loss: 0.0711 - accuracy: 0.9759
Epoch 3/10
313/313 [==============================] - 2s 8ms/step - loss: 0.0485 - accuracy: 0.9856
Epoch 4/10
313/313 [==============================] - 2s 7ms/step - loss: 0.0303 - accuracy: 0.9902
Epoch 5/10
313/313 [==============================] - 2s 8ms/step - loss: 0.0317 - accuracy: 0.9901
Epoch 6/10
313/313 [==============================] - 2s 7ms/step - loss: 0.0238 - accuracy: 0.9925
Epoch 7/10
313/313 [==============================] - 2s 8ms/step - loss: 0.0166 - accuracy: 0.9950
Epoch 8/10
313/313 [==============================] - 2s 8ms/step - loss: 0.0196 - accuracy: 0.9938
Epoch 9/10
313/313 [==============================] - 2s 7ms/step - loss: 0.0187 - accuracy: 0.9938
Epoch 10/10
313/313 [==============================] - 2s 7ms/step - loss: 0.0177 - accuracy: 0.9949

In [10]:
img = keras.preprocessing.image.load_img(
    "/home/karol/python_projekty/cv2/sudoku/data/train/2/img003-00005.png",
    target_size=(128, 128),
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, -2)  # Create batch axis

predictions = model.predict(img_array)
print(argmax(predictions))

ValueError: in user code:

    File "/home/karol/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "/home/karol/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/karol/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "/home/karol/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1751, in predict_step
        return self(x, training=False)
    File "/home/karol/.local/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/karol/.local/lib/python3.10/site-packages/keras/engine/input_spec.py", line 264, in assert_input_compatibility
        raise Va